# DSCI 511: Data acquisition and pre-processing<br>Chapter 7: Building and Maintaining a Robust Acquisition Stream

## Exercises
Note: numberings refer to the main notes.

#### 7.1.1.2 Exercise: Understanding API rate limits
Read each of the above API docs and describe the how much API usage is allowed per day from each platform for a given app. Do all apps get the same bandwidth? What methods/metrics do the platforms use to determine limits and overuse? How should an app be constructed to maximize data access?

_Response._

#### 7.1.2.3 Exercise: robots.txt
Take a look at the robots file for each of `facebook.com` and `amazon.com`. Determine and discuss any allowances/disallowances for bots that you might create to crawl these sites. Do you infer any cultural differences around data sharing and access between these companys and also with Twitter?

_Response._

#### 7.2.1.2 Exercise: understanding a crontab for a recurrent, whole-site data access application
Gutenberg is an open data repository, so we should be able to download all of its data!  To start, let's review the robots file on Project Gutenberg's website:
- http://www.gutenberg.org/robots.txt

What do you notice about this file. Is anyone allowed to crawl the site? Do you think Gutenberg uses the newer, big tech rules? How frequently can we make requests?

Use the `robotexclusionrulesparser` module from Section 7.1.2.4 to determine if we can access a given data file. Use the URL for the text copy of Moby dick: 
- https://www.gutenberg.org/files/2701/2701-0.txt

Following the above, review the instructions on mirroring the repository:
- https://www.gutenberg.org/wiki/Gutenberg:Mirroring_How-To

and explain why Gutenberg requests using the `rsync` command-line utility to copy its data. Can you decode the two presented crontab patterns?

_Response._

In [ ]:
## place code here

#### 7.3.2.2 Exercise: a script restarter using psutil that also kills zombies
Rewrite `check_process(name)` above by using psutil to 1) obtain process names more easily without regex, and use this 2) to restart our dummy process if it's finished after 3 or fewer passes in the while loop, and kill it if it's still running after 4 or more passes. 

In [11]:
import os, re, psutil, datetime

def check_process(process_name):
    ## review the current running processes
    ## processes = os.popen("ps -A").read().split("\n")
    processes = list(psutil.process_iter())
    
    ## Get the process names
    ## Splits up the row by whitespace, then looks at the last element (the name)
    ## process_names = [re.split('\s+', row.strip())[-1] for row in processes]
    process_names = [(process.cmdline(), process) for process in processes if process.status() != "zombie"]
    print(process_names)

    ## create a boolean valued list indicating if the process is running
    is_running = [name for name in process_names if re.search(process_name, name[0])]

    ## Let the user know which processes are running
    return(is_running)

In [12]:
## create a log file for our script-monitoring code
logfile = 'restarter.log'
open(logfile, 'w').close()

## set the process name for what we'd like to watch/kill
name = 'dummy.py'

## perform the initial execution
current_time = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d-%H-%M")
status = os.system("nohup python3 ./dummy.py &")
with open(logfile, 'a') as f:  # Open the logfile
    f.writelines("Started the process " + name +"\n") 

## keep track of the number of passes we've made wheil watching
passes = 0
while 1:
    passes += 1
    current_time = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d-%H-%M")
    instances = check_process(name)
    
    if instances:  # The stream is working fine
        if passes == 4: ## process has been around too long, kill it!
            for instance in instances:
                instance[1].kill()
            with open(logfile, 'a') as f:  # Open the logfile
                f.writelines(current_time + " : The process " + name + " was running for too long, so I killed it!\n")  # Write that all is well
                f.writelines("shutting down the script watcher now...\n\n")
            break ## break out of the while loop if we decided to kill the old process(es)
        else: ## process still ok, don't kill, just log status
            with open(logfile, 'a') as f:  # Open the logfile
                f.writelines(current_time + " : The process " + name + " is running \n")  # Write that all is well
                f.writelines("sleeping for another 10 seconds...\n\n")
    else:  # Stream is down
        with open(logfile, 'a') as f:  # Open the logfile
            f.writelines(current_time + " : The process " + name + " died! Restarting... \n") # Note error
        status = os.system("nohup python3 ./dummy.py &")  # Finally, restart the stream
        ## for the notebook---break if the code got restarted
        break
    ## sleep for another 2 seconds
    time.sleep(2)

AccessDenied: psutil.AccessDenied (pid=1, name='launchd')